In [4]:
import numpy as np
import matplotlib.pyplot as plt

### Exercise 1

For the Brock and Mirman model, find the value of A in the policy function. Show that your value is correct.

For this case find an algebraic solution for the policy function $k_{t+1} = \Phi(k_t, z_t)$. Sources for hints are Stokey et al. (1989, exercise 2.2, p. 12) and Sargent (1987, exercise 1.1, p. 47).

#### Brock and Mirman's Model

Households solve the following dynamic program:

$$ V(K_t, z_t) =  max_{K_{t+1}}ln(e^{z_t}K_t^{\alpha} - K_{t+1}) + \beta E_t\{V(K_{t+1}, z_{t+1})\}$$

The associated Euler equaation is:

$$ \frac{1}{e^{z_t}K_t^{\alpha} - K_{t+1}} =  \beta E_t\{\frac{\alpha e^{z_{t+1}}K_{t+1}^{\alpha-1}}{e^{z_{t+1}}K_{t+1}^{\alpha} - K_{t+2}}\}$$

The law of motion for z is:

$$z_{t+1} = \rho z_t + \epsilon_t \text{;   }\epsilon_t \approx i.i.d(0,\sigma^2)$$

Verify that the policy function is of the following form:

$$K_{t+1} = Ae^{z_t}K_t^{\alpha}$$

In order to determine the policy function, we need to find the value of A as a function of the model's parameters. 

## Exercise 2

For the model in section 3 of the notes consider the following functional forms:

$$u(c_t, l_t) = ln c_t + a \text{ }ln(1-l_t)$$

$$F(K_t, L_t, z_t) = e^{z_t}K_t^{\alpha}K_t^{1-\alpha}$$

Write out all the caracterizing equations for the model using these functional forms. Can you use the same tricks as in Exercise 1 to solve for the policy function in this case? Why or why not?


### Exercise 5

In [2]:
## Lets estimate the steady state values

In [1]:
from sympy import var, exp, solve

k, l = var('k l')
c, w, r, T = var('c w r T')
z = var('z')
gamma, beta, alpha, sigma, tau = var('gamma beta alpha sigma tau')

In [5]:
E1 = (1-tau)*(w + (r-sigma)*k) +T - c
E2 = beta*((r-sigma)*(1-tau)+1) -1
E3 = (c**(-gamma) - 1/(1-gamma))*w*(1-tau)
E4 = r - alpha*k**(alpha-1)*exp(-alpha*z)*exp(z)
E5 = w - k**(alpha)*(1-alpha)*exp(-alpha*z)*exp(z)
E6 = T -tau*(w+(r-sigma)*k)

In [6]:
for i in [E1, E2, E3, E4, E5, E6]:
    print(i)

T - c + (-tau + 1)*(k*(r - sigma) + w)
beta*((r - sigma)*(-tau + 1) + 1) - 1
w*(-tau + 1)*(-1/(-gamma + 1) + c**(-gamma))
-alpha*k**(alpha - 1)*exp(z)*exp(-alpha*z) + r
-k**alpha*(-alpha + 1)*exp(z)*exp(-alpha*z) + w
T - tau*(k*(r - sigma) + w)


In [7]:
import numpy as np
from scipy.optimize import root
gamma = 2.5
beta = 0.98
alpha = 0.40
sigma = 0.1
tau = 0.05
z = 0
def func_p5(x):
    k = x[0]
    r = x[1]
    c = x[2]
    w = x[3]
    T = x[4]

    
    E1 = (1-tau)*(w + (r-sigma)*k) +T - c
    E2 = beta*((r-sigma)*(1-tau)+1) -1
    E4 = r - alpha*(k**(alpha-1))*(np.exp(z)**(1-alpha))
    E5 = w - k**(alpha)*(1-alpha)*(np.exp(z)**(-alpha))*np.exp(z)
    E6 = T -tau*(w+(r-sigma)*k)
    
    return [E1, E2, E4, E5, E6] 

k_true = (((1-beta)/beta*(1/(1-tau)) + sigma)/((np.exp(z*(1-alpha))*alpha)))**(1/(alpha-1))

In [8]:
sols = root(func_p5, np.ones(5)*.5)

In [9]:
sols.x

array([7.28749795, 0.12148228, 1.48450482, 1.32795277, 0.07422524])

In [55]:
k_true

7.287497950692989

### Exercise 6

In [3]:
from sympy import var, exp, solve, diff, lambdify

cVar, lVar, gammaVar, xiVar, aVar = var('cVar lVar gammaVar xiVar aVar')
u = (cVar**(1-gammaVar) -1)/(1-gammaVar) + aVar*((1-lVar)**(1-xiVar) -1)/(1-xiVar)


KVar, LVar, zVar, alphaVar = var('KVar LVar zVar alphaVar')
F = (KVar**alphaVar)*(LVar*exp(zVar))**(1-alphaVar)

du_dc = diff(u, cVar)
du_dl = diff(u, lVar)
du_dc_l = lambdify([cVar, lVar, gammaVar, xiVar, aVar], du_dc, 'numpy')
du_dl_l = lambdify([cVar, lVar, gammaVar, xiVar, aVar], du_dl, 'numpy')

dF_dK = diff(F, KVar)
dF_dL = diff(F, LVar)
dF_dK_l = lambdify([KVar, LVar, zVar, alphaVar], dF_dK, 'numpy')
dF_dL_l = lambdify([KVar, LVar, zVar, alphaVar], dF_dL, 'numpy')

In [15]:
dF_dK

KVar**alphaVar*alphaVar*(LVar*exp(zVar))**(1 - alphaVar)/KVar

In [9]:
import numpy as np
from scipy.optimize import root

gamma = 2.5
beta = 0.98
alpha = 0.40
sigma = 0.1
tau = 0.05
a = 0.5
z = 0
xi = 2.5

def general_steady_state(x):
    k = x[0]
    l = x[1]
    r = x[2]
    c = x[3]
    w = x[4]
    T = x[5]
    
    E1 = (1-tau)*(w*l + (r-sigma)*k) +T - c
    E2 = du_dc_l(c, l, gamma, xi, a)*(1-beta*((r-sigma)*(1-tau)+1))
    E3 = du_dl_l(c, l, gamma, xi, a)+du_dc_l(c, l, gamma, xi, a)*w*(1-tau)
    E4 = r - dF_dK_l(k, l, z, alpha)
    E5 = w - dF_dL_l(k, l, z, alpha)
    E6 = T -tau*(w*l+(r-sigma)*k)
    
    return [E1, E2, E3, E4, E5, E6] 

sols = root(general_steady_state, np.ones(6)*.5)

In [16]:
sols

    fjac: array([[-5.88480630e-02,  6.36384917e-03,  7.57603368e-02,
        -1.92519556e-01,  9.76567154e-01,  3.09726561e-03],
       [-8.64845371e-02, -2.70421991e-03,  9.92952759e-01,
         1.05241751e-02, -8.01651602e-02,  4.55181770e-03],
       [ 5.98116818e-01, -7.54615788e-01,  5.44765097e-02,
         2.47999946e-01,  8.57242220e-02, -3.14798317e-02],
       [ 7.91151844e-01,  5.95567644e-01,  7.30004291e-02,
        -1.17526721e-01,  1.49725770e-02, -3.48867764e-03],
       [ 7.29357938e-02, -2.73619932e-01,  2.78674952e-04,
        -9.23518574e-01, -1.76504905e-01,  1.89134572e-01],
       [-8.52835616e-03, -3.06352077e-02,  2.89121546e-03,
        -1.86070958e-01, -3.41077658e-02, -9.81424807e-01]])
     fun: array([ 1.73359105e-10,  2.78902932e-10, -3.31712435e-10,  2.48078974e-10,
       -5.71493519e-10, -9.12415976e-12])
 message: 'The solution converged.'
    nfev: 32
     qtf: array([-8.93540917e-09, -4.67206628e-08, -3.95056867e-09, -2.86224593e-09,
       -7.8993